# Reading Notes

In [38]:
import numpy as np

### Section 2 - Comparisons on two Examples

Below are implemented all algorithms mentioned in chapter 1. They are all realted to the product function
$$f:\mathbb{R}^n \to \mathbb{R}, \quad f(x_1,...,x_n) \mapsto \prod_{i=1}^n x_i$$
which has derivative
$$g := f^\prime:\mathbb{R}^n \to \mathbb{R^n}, \quad f(x_1,...,x_n) \mapsto \sum_{j=1}^n e_j\prod_{i\neq j}^n x_i.$$


In [37]:
# Evaluation of Product [p.7] 
# note idexing must be shifted back by one since the paper indexes from 1, but python indexes from 0

# setup
x = [1,3,3,2,7]
n = len(x)
x += [None] * n

# START OF ALGORITHM
x[n] = x[0]
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1]
y = x[2*n-1]
# END OF ALGORITHM

print(x)
print(y)
print(1*3*3*2*7)
print("yay! the algorithm works!")

[1, 3, 3, 2, 7, 1, 3, 9, 18, 126]
126
126
yay, the algorithm works!


In [49]:
# Forward Differentiation of Product [p.7] 

x = [2,3,5]
n = len(x)
x += [None] * n
dx = [None] * (2 * n) # defining unnecessarily long array so that inexing is more consistent with paper.

def e(i):
    return np.eye(1, n, i)

# START OF ALGORITHM
x[n] = x[0]
dx[n] = e(0)
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1]
    dx[i] = x[i - 1] * e(i - n) + x[i - n] * dx[i - 1]
y = x[2*n-1]
g = dx[2*n-1]
# END OF ALGORITHM

print(y)
print(g)
print("yay! algorithm works as expected!")

30
[[15. 10.  6.]]


In [58]:
# Reverse Differentiation of Product [p.8] 

x = [2,3,5]
n = len(x)
x += [None] * n
x_bar = [None] * (2 * n) # defining unnecessarily long array so that inexing is more consistent with paper.

def e(i):
    return np.eye(1, n, i)

# START OF ALGORITHM
x[n] = x[0]
for i in range(n+1, 2*n):
    x[i] = x[i - n] * x[i - 1] # forward step
y = x[2 * n - 1]

x_bar[2 * n - 1] = 1
for i in range(2*n - 1, n, -1):
    x_bar[i - 1] = x_bar[i] * x[i - n] # reverse step
    x_bar[i - n] = x_bar[i] * x[i - 1]
x_bar[0] = x_bar[n]
g = x_bar[0:n]
# END OF ALGORITHM

print("y =", y)
print(g)
print("yay! algorithm works as expected!")

y = 30
[15, 10, 6]
yay! algorithm works as expected!


### Section 3 - Automatic Differentiation on Composite Functions
